## 设置

In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = ""
os.environ["OPENAI_API_BASE"] = ""

from dotenv import load_dotenv
load_dotenv(override=True)

True

### 补全

In [ ]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0)

In [ ]:
res1 = llm.predict("1 + 1 = ", stop=("="))
res2 = llm("1 + 1 = ", stop = ["="])
res3 = llm.generate(["1 + 1 = "], stop = ["="]).generations[0][0].text

In [ ]:
llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0, n=2)
llm_result = llm.generate([f"{i}**2 =" for i in range(1, 11)], stop="\n")
print(len(llm_result.generations))
print(len(llm_result.generations[0]))

In [ ]:
llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0, n=2)
llm_result = llm.generate([f"{i}**2 =" for i in range(1, 11)], stop="\n", n=1)

In [ ]:
llm_result.json()

### 聊天

In [4]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0)

In [9]:
messages = [
    SystemMessage(content="你是一名翻译员，将中文翻译成英文"),
    HumanMessage(content="你好世界")
]
chat(messages)

AIMessage(content='Hello world.', additional_kwargs={}, example=False)

In [13]:
chat.generate([messages, messages])

LLMResult(generations=[[ChatGeneration(text='Hello world.', generation_info=None, message=AIMessage(content='Hello world.', additional_kwargs={}, example=False))], [ChatGeneration(text='Hello world.', generation_info=None, message=AIMessage(content='Hello world.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 80, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo'}, run=RunInfo(run_id=UUID('fffa5a38-c738-4eef-bdc4-0071511d1422')))

## prompt

In [ ]:
from langchain import PromptTemplate

template = """
我希望你担任顾问，帮忙为公司想名字。
这个公司生产{product}，有什么好名字？
"""

prompt_template = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt_template.format(product="运动衬衫")

In [ ]:
prompt_template = PromptTemplate.from_template(template)
prompt_template.input_variables

In [ ]:
from langchain.prompts import load_prompt
prompt_template.save("prompt.json")
prompt_template = load_prompt("prompt.json")

## Chain

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

chat = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0)
prompt = PromptTemplate(
    input_variables=["input"],
    template="""
    将给定的字符串进行大小写转换。
    例如：
    输入： ABCdef
    输出： abcDEF
    
    输入： AbcDeF
    输出： aBCdEf
    
    输入： {input}
    输出： 
    """,
)

chain = LLMChain(llm=chat, prompt=prompt)
print(chain.run("HeLLo"))
print(chain.run({"input": "HeLLo"}))


hEllO
hEllO


In [3]:
chain_verbose = LLMChain(llm=chat, prompt=prompt, verbose=True)
print(chain_verbose.run({"input": "HeLLo"}))



> Entering new  chain...
Prompt after formatting:

    将给定的字符串进行大小写转换。
    例如：
    输入： ABCdef
    输出： abcDEF
    
    输入： AbcDeF
    输出： aBCdEf
    
    输入： HeLLo
    输出： 
    

> Finished chain.
hEllO


In [ ]:
from langchain.chains import SimpleSequentialChain

chat = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0)
prompt1 = PromptTemplate(
    input_variables=["base"], template="{base}的平方是： "
)
chain1 = LLMChain(llm=chat, prompt=prompt1)

prompt2 = PromptTemplate(input_variables=["input"], template="将{input}写成罗马数字是：")
chain2 = LLMChain(llm=chat, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
overall_chain.run(3)


In [ ]:
import os
# import openai
# openai.api_type = "azure"
# openai.api_base = "https://ai-jambo.openai.azure.com/"
# openai.api_version = "2023-03-15-preview"

os.environ["OPENAI_API_KEY"] = "bdc947a18c07495593efdfa0fe63a5ce"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_ORGANIZATION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://ai-jambo.openai.azure.com/"

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
chat = ChatOpenAI(temperature=0,deployment_id="gpt-35-turbo")

In [ ]:
messages = [
    SystemMessage(content="你是一名翻译员，将中文翻译成英文"),
    HumanMessage(content="你好世界")
]
chat(messages)